# GHCND Inventory Data Prep
#### Description:
This notebook is used to prep the data contained in ghcnd-inventory.txt. Use noaa-daily-retrieve-files-ftp.ipynb to download this file among other metadta files used in the NOAA Daily Weather Project. 

System of Origin Data Source: https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-inventory.txt

#### Data Prep Operations:
- define english column names
- save to .csv file (tab separated)

#### Created by:
Nate Muth <br>
nmuth87@gmail.com

#### Created on:
8/5/2018

#### Changelog:
8/5/2018 - Initial Create Date<br>

In [2]:
import pandas as pd

In [11]:
# read inventory.txt into DataFrame
inventoryDF = pd.read_fwf('ghcnd-inventory.txt', header=None, delimiter=' '
                         , widths=[12,9,10,5,5,5]
                         , names=['StationID', 'Latitude', 'Longitude', 'Element',
                                 'FirstYear', 'LastYear']
                         #, dtypes={'WMO_ID':object}
                         )

In [12]:
inventoryDF[inventoryDF['StationID']=='USW00014942'].head()

,StationID,Latitude,Longitude,Element,FirstYear,LastYear
583588,USW00014942,41.3103,-95.8992,TMAX,1948,2018
583589,USW00014942,41.3103,-95.8992,TMIN,1948,2018
583590,USW00014942,41.3103,-95.8992,PRCP,1948,2018
583591,USW00014942,41.3103,-95.8992,SNOW,1948,2018
583592,USW00014942,41.3103,-95.8992,SNWD,1948,2018


In [14]:
# Write file to local directory
inventoryDF.to_csv('ghcnd-inventory-cleansed.csv',sep='\t')